In [9]:
import os
import sys
sys.path.insert(0, os.path.abspath('./source/etl'))

from pyspark.sql import SparkSession
from SparkDBUtils import SparkDB
from delta import DeltaTable
import datetime as dt
import pyspark.sql
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StructType, StructField, IntegerType, TimestampType, StringType, FloatType
from pyspark.sql.window import Window

sparkDB = SparkDB()
spark = sparkdb.spark

In [16]:
simple_schema = StructType([
        StructField("date", DateType(), True),
        StructField("product", StringType(), True),
        StructField("product_id", StringType(), True),
        StructField("brand", StringType(), True),
        StructField("price", FloatType(), True),
        StructField("categories", StringType(), True),
        StructField("unit_price", FloatType(), True),
        StructField("units", StringType(), True),
        StructField("discount", FloatType(), True),
        StructField("ts_load", TimestampType(), True)
    ])

dataset = spark.read.option("delimiter", ";") \
    .csv("../../dataset/dataset.csv", schema=simple_schema, header=True)\
    .withColumn("categoria", f.lit("INIT_CATEGORIA"))

In [20]:
year = 2023

# tabkla de productos para hacer el lookup
product_dim_db = sparkDB.read_table("producto_dim").select(["product_id", "id_producto"])

# tabla de fechas para hacer el lookup
date_dim_db = sparkDB.read_table("date_dim").select(["date", "id_date", "year"])

# Cogemos los hechos del dataset de la particion
producto_dia_fact_new = dataset\
            .join(product_dim_db.alias("p"), "product_id", "left")\
            .join(date_dim_db.alias("d"), "date", "left")\
            .select(["price",
                     "unit_price",
                     "discount",
                     "id_producto",
                     "id_date",
                     "d.year"])\
            .where(f"year = {year}")

In [21]:
producto_dia_fact_new.where("product_id = 119675").show()

+-----+----------+--------+-----------+-------+----+
|price|unit_price|discount|id_producto|id_date|year|
+-----+----------+--------+-----------+-------+----+
| 1.43|      1.43|    null|        344|     26|2023|
| 1.43|      1.43|    null|        344|     27|2023|
| 1.43|      1.43|    null|        344|     28|2023|
| 1.43|      1.43|    null|        344|     29|2023|
| 1.43|      1.43|    null|        344|     30|2023|
| 1.43|      1.43|    null|        344|     31|2023|
| 1.43|      1.43|    null|        344|     32|2023|
| 1.43|      1.43|    null|        344|     33|2023|
| 1.43|      1.43|    null|        344|     34|2023|
| 1.43|      1.43|    null|        344|     35|2023|
| 1.43|      1.43|    null|        344|     36|2023|
| 1.43|      1.43|    null|        344|     37|2023|
| 1.43|      1.43|    null|        344|     38|2023|
| 1.43|      1.43|    null|        344|     39|2023|
| 1.43|      1.43|    null|        344|     40|2023|
| 1.43|      1.43|    null|        344|     41